In [1]:
# Imports
import os
import pandas as pd
import numpy as np
from IPython.display import display

# Definitions
pd.set_option('display.float_format', lambda x: '%.3f' % x)
%matplotlib inline
FOLDS = 5
N_JOBS = 6
SEED = 2016
TRAIN_SPLIT = 0.8

DATA = "April"

In [2]:
# Get data
train = pd.read_csv("raw_data/input_train.csv", sep = ";")
test = pd.read_csv("raw_data/input_test.csv", sep = ";")
y = pd.read_csv("raw_data/output_train.csv", sep = ";")
print("train : " + str(train.shape))
print("test : " + str(test.shape))
print("y : " + str(y.shape))

alldata = pd.concat([train, test], axis = 0)
display(alldata.head())
display(y.head())

# Rename and change order of columns
alldata.columns = ["Id", "Id_bat", "Time", "Sensor1", "Weather1", "Sensor2", "Sensor3", "Weather2"]
alldata = pd.concat([alldata.Id, alldata.Id_bat, alldata.Time, alldata.Sensor1, alldata.Sensor2, alldata.Sensor3, 
                     alldata.Weather1, alldata.Weather2], axis = 1)
display(alldata.head())

train = alldata.iloc[:train.shape[0], :]
test = alldata.iloc[train.shape[0]: , :]

# Differentiate numerical features (minus the target) and categorical features
cat_features = alldata.select_dtypes(include = ["object"]).columns
print("Categorical features : " + str(len(cat_features)))
num_features = alldata.select_dtypes(exclude = ["object"]).columns
print("Numerical features : " + str(len(num_features)))

train : (11708, 8)
test : (2429, 8)
y : (11708, 6)


,Id,Id_bat,Time,x1,x2,x3,x4,x5
0,0,1,2016-01-01 00:00:00,1.500,8.100,22.000,380.183,20.000
1,1,1,2016-01-01 01:00:00,1.600,8.200,21.900,378.100,120.000
2,2,1,2016-01-01 02:00:00,1.400,7.900,21.900,374.983,120.000
3,3,1,2016-01-01 03:00:00,1.600,6.900,21.800,376.017,120.000
4,4,1,2016-01-01 04:00:00,1.400,6.300,21.800,376.017,120.000


,Id,y1,y2,y3,y4,y5
0,0,110.500,40.900,0.000,48,13.940
1,1,110.800,38.600,0.000,48,20.910
2,2,112.600,37.000,0.000,48,20.910
3,3,110.900,37.600,0.000,48,41.820
4,4,112.500,38.400,0.000,52,69.700


,Id,Id_bat,Time,Sensor1,Sensor2,Sensor3,Weather1,Weather2
0,0,1,2016-01-01 00:00:00,1.500,22.000,380.183,8.100,20.000
1,1,1,2016-01-01 01:00:00,1.600,21.900,378.100,8.200,120.000
2,2,1,2016-01-01 02:00:00,1.400,21.900,374.983,7.900,120.000
3,3,1,2016-01-01 03:00:00,1.600,21.800,376.017,6.900,120.000
4,4,1,2016-01-01 04:00:00,1.400,21.800,376.017,6.300,120.000


Categorical features : 1
Numerical features : 7


In [3]:
# Imputing NAs
# Look at missing values
print("NAs : " + str(alldata.isnull().values.sum()))
print(alldata.isnull().sum())

# Imput NAs as median of that variable for this building
def replaceBySensor1Medians(group):
    group = group.fillna(train["Sensor1"].groupby(train["Id_bat"]).median()[group.name])
    return group
def replaceBySensor2Medians(group):
    group = group.fillna(train["Sensor2"].groupby(train["Id_bat"]).median()[group.name])
    return group
def replaceBySensor3Medians(group):
    group = group.fillna(train["Sensor3"].groupby(train["Id_bat"]).median()[group.name])
    return group
def replaceByWeather1Medians(group):
    group = group.fillna(train["Weather1"].groupby(train["Id_bat"]).median()[group.name])
    return group
def replaceByWeather2Medians(group):
    group = group.fillna(train["Weather2"].groupby(train["Id_bat"]).median()[group.name])
    return group

train.loc[:, "Sensor1"] = train.loc[:, "Sensor1"].groupby(train["Id_bat"]).transform(replaceBySensor1Medians)
test.loc[:, "Sensor1"] = test.loc[:, "Sensor1"].groupby(train["Id_bat"]).transform(replaceBySensor1Medians)
train.loc[:, "Sensor2"] = train.loc[:, "Sensor2"].groupby(train["Id_bat"]).transform(replaceBySensor2Medians)
test.loc[:, "Sensor2"] = test.loc[:, "Sensor2"].groupby(train["Id_bat"]).transform(replaceBySensor2Medians)
train.loc[:, "Sensor3"] = train.loc[:, "Sensor3"].groupby(train["Id_bat"]).transform(replaceBySensor3Medians)
test.loc[:, "Sensor3"] = test.loc[:, "Sensor3"].groupby(train["Id_bat"]).transform(replaceBySensor3Medians)
train.loc[:, "Weather1"] = train.loc[:, "Weather1"].groupby(train["Id_bat"]).transform(replaceByWeather1Medians)
test.loc[:, "Weather1"] = test.loc[:, "Weather1"].groupby(train["Id_bat"]).transform(replaceByWeather1Medians)
train.loc[:, "Weather2"] = train.loc[:, "Weather2"].groupby(train["Id_bat"]).transform(replaceByWeather2Medians)
test.loc[:, "Weather2"] = test.loc[:, "Weather2"].groupby(train["Id_bat"]).transform(replaceByWeather2Medians)

print("After NA imputing, remaining NAs : " + str(alldata.isnull().values.sum()))
print(alldata.shape)

NAs : 745
Id            0
Id_bat        0
Time          0
Sensor1      47
Sensor2      47
Sensor3     321
Weather1    149
Weather2    181
dtype: int64


C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


After NA imputing, remaining NAs : 0
(14137, 8)


In [4]:
# Only keep data from April onward
train = alldata.iloc[:train.shape[0], :]
test = alldata.iloc[train.shape[0]: , :]

temp_train = pd.concat([train, y], axis = 1)
temp_train["Date"], temp_train["Hour"] = temp_train["Time"].str.split(" ", 1).str
temp_train["Year"], temp_train["Month"], temp_train["DayOfMonth"] = temp_train["Date"].str.split("-").str
temp_train["Month"] = temp_train["Month"].astype(int)
temp_train = temp_train.loc[temp_train.Month > 3]
temp_train = temp_train.drop(["Date", "Year", "DayOfMonth", "Hour", "Month"], axis = 1) 

train = temp_train.iloc[:, :train.shape[1]]
y = temp_train.iloc[: , train.shape[1]:]
display(train.head())
print(train.shape)
display(y.head())
print(y.shape)

alldata = pd.concat([train, test], axis = 0)


,Id,Id_bat,Time,Sensor1,Sensor2,Sensor3,Weather1,Weather2
2183,2183,1,2016-04-01 00:00:00,3.900,23.100,592.667,7.600,20.000
2184,2184,1,2016-04-01 01:00:00,4.000,23.000,581.233,7.400,20.000
2185,2185,1,2016-04-01 02:00:00,3.600,22.900,568.733,7.000,20.000
2186,2186,1,2016-04-01 03:00:00,4.200,22.800,549.967,6.100,20.000
2187,2187,1,2016-04-01 04:00:00,3.800,22.700,526.033,5.800,20.000


(2976, 8)


,Id,y1,y2,y3,y4,y5
2183,2183,114.800,0.000,0.000,60,90.610
2184,2184,112.000,0.000,0.000,64,83.640
2185,2185,112.100,0.000,0.000,60,90.610
2186,2186,111.200,0.000,0.000,64,83.640
2187,2187,113.100,0.000,0.000,148,90.610


(2976, 6)


In [5]:
# Encode cat features
for feat in cat_features :
    alldata[feat] = pd.factorize(alldata[feat], sort = True)[0]
print(alldata.shape)

train = alldata.iloc[:train.shape[0], :]
test = alldata.iloc[train.shape[0]: , :]

(5405, 8)


In [6]:
# Prepare one training set for each target variable, removing the correct rows
print("Important remark: for some buildings, some consumptions are not used to heat/cool rooms. In this case, all data of \
the corresponding output are set to 0 (zero) and this output has to be ignored (you do not have to predict it). This \
is for instance the case for output y3 for building with Id_bat 1.")
print("-----")
temp_train = pd.concat([train, y], axis = 1)
print(temp_train.groupby(["Id_bat"]).y1.sum())
print(temp_train.groupby(["Id_bat"]).y2.sum())
print(temp_train.groupby(["Id_bat"]).y3.sum())
print(temp_train.groupby(["Id_bat"]).y4.sum())
print(temp_train.groupby(["Id_bat"]).y5.sum())
print("-----")
print("For Bat1 : y3 always zero")
print("For Bat2 : y3 always zero")
print("For Bat3 : y2 and y4 always zero")
print("For Bat4 : y3 always zero")

train1 = train
train2 = train.drop(train[train.Id_bat == 3].index, axis = 0)
train3 = train.drop(train[train.Id_bat != 3].index, axis = 0)
train4 = train.drop(train[train.Id_bat == 3].index, axis = 0)
train5 = train

y_1 = y[["Id", "y1"]]
y_2 = y.drop(y[train.Id_bat == 3].index, axis = 0)[["Id", "y2"]]
y_3 = y.drop(y[train.Id_bat != 3].index, axis = 0)[["Id", "y3"]]
y_4 = y.drop(y[train.Id_bat == 3].index, axis = 0)[["Id", "y4"]]
y_5 = y[["Id", "y5"]]
print("-----")
print("train1.shape" + str(train1.shape))
print("train2.shape" + str(train2.shape))
print("train3.shape" + str(train3.shape))
print("train4.shape" + str(train4.shape))
print("train5.shape" + str(train5.shape))

print("y_1.shape" + str(y_1.shape))
print("y_2.shape" + str(y_2.shape))
print("y_3.shape" + str(y_3.shape))
print("y_4.shape" + str(y_4.shape))
print("y_5.shape" + str(y_5.shape))


Important remark: for some buildings, some consumptions are not used to heat/cool rooms. In this case, all data of the corresponding output are set to 0 (zero) and this output has to be ignored (you do not have to predict it). This is for instance the case for output y3 for building with Id_bat 1.
-----
Id_bat
1   107149.050
2    74951.900
3        0.000
4    96003.300
Name: y1, dtype: float64
Id_bat
1        0.000
2   114759.400
3        0.000
4   231457.055
Name: y2, dtype: float64
Id_bat
1       0.000
2       0.000
3   15276.100
4       0.000
Name: y3, dtype: float64
Id_bat
1    118508
2         0
3         0
4    140212
Name: y4, dtype: int64
Id_bat
1   101538.960
2    60300.000
3   103290.000
4    28680.000
Name: y5, dtype: float64
-----
For Bat1 : y3 always zero
For Bat2 : y3 always zero
For Bat3 : y2 and y4 always zero
For Bat4 : y3 always zero
-----
train1.shape(2976, 8)
train2.shape(2232, 8)
train3.shape(744, 8)
train4.shape(2232, 8)
train5.shape(2976, 8)
y_1.shape(2976, 2)
y_

In [7]:
# Split data for final performance check
# Time series -> split by time
X_train1 = train1[0:int(train1.shape[0] * TRAIN_SPLIT)]
X_train2 = train2[0:int(train2.shape[0] * TRAIN_SPLIT)]
X_train3 = train3[0:int(train3.shape[0] * TRAIN_SPLIT)]
X_train4 = train4[0:int(train4.shape[0] * TRAIN_SPLIT)]
X_train5 = train5[0:int(train5.shape[0] * TRAIN_SPLIT)]

X_test1 = train1[int(train1.shape[0] * TRAIN_SPLIT):]
X_test2 = train2[int(train2.shape[0] * TRAIN_SPLIT):]
X_test3 = train3[int(train3.shape[0] * TRAIN_SPLIT):]
X_test4 = train4[int(train4.shape[0] * TRAIN_SPLIT):]
X_test5 = train5[int(train5.shape[0] * TRAIN_SPLIT):]

y_train1 = y_1[0:int(train1.shape[0] * TRAIN_SPLIT)]
y_train2 = y_2[0:int(train2.shape[0] * TRAIN_SPLIT)]
y_train3 = y_3[0:int(train3.shape[0] * TRAIN_SPLIT)]
y_train4 = y_4[0:int(train4.shape[0] * TRAIN_SPLIT)]
y_train5 = y_5[0:int(train5.shape[0] * TRAIN_SPLIT)]

y_test1 = y_1[int(train1.shape[0] * TRAIN_SPLIT):]
y_test2 = y_2[int(train2.shape[0] * TRAIN_SPLIT):]
y_test3 = y_3[int(train3.shape[0] * TRAIN_SPLIT):]
y_test4 = y_4[int(train4.shape[0] * TRAIN_SPLIT):]
y_test5 = y_5[int(train5.shape[0] * TRAIN_SPLIT):]

# Add Id_bat field to y_tests
y_test1.Id_bat = X_test1.Id_bat
y_test2.Id_bat = X_test2.Id_bat
y_test3.Id_bat = X_test3.Id_bat
y_test4.Id_bat = X_test4.Id_bat
y_test5.Id_bat = X_test5.Id_bat

In [8]:
# Start training on 2nd day
train1 = train1.loc[train1.Id > 23]
train2 = train2.loc[train2.Id > 23]
train3 = train3.loc[train3.Id > 23]
train4 = train4.loc[train4.Id > 23]
train5 = train5.loc[train5.Id > 23]

y_1 = y_1.loc[y_1.Id > 23]
y_2 = y_2.loc[y_2.Id > 23]
y_3 = y_3.loc[y_3.Id > 23]
y_4 = y_4.loc[y_4.Id > 23]
y_5 = y_5.loc[y_5.Id > 23]

X_train1 = X_train1.loc[X_train1.Id > 23]
X_train2 = X_train2.loc[X_train2.Id > 23]
X_train3 = X_train3.loc[X_train3.Id > 23]
X_train4 = X_train4.loc[X_train4.Id > 23]
X_train5 = X_train5.loc[X_train5.Id > 23]

y_train1 = y_train1.loc[y_train1.Id > 23]
y_train2 = y_train2.loc[y_train2.Id > 23]
y_train3 = y_train3.loc[y_train3.Id > 23]
y_train4 = y_train4.loc[y_train4.Id > 23]
y_train5 = y_train5.loc[y_train5.Id > 23]


In [9]:
print(train1.shape)
print(y_1.shape)
print(train2.shape)
print(y_2.shape)
print(train3.shape)
print(y_3.shape)
print(train4.shape)
print(y_4.shape)
print(train5.shape)
print(y_5.shape)
print("----")
print(X_train1.shape)
print(y_train1.shape)
print(X_train2.shape)
print(y_train2.shape)
print(X_train3.shape)
print(y_train3.shape)
print(X_train4.shape)
print(y_train4.shape)
print(X_train5.shape)
print(y_train5.shape)
print("----")
print(X_test1.shape)
print(y_test1.shape)
print(X_test2.shape)
print(y_test2.shape)
print(X_test3.shape)
print(y_test3.shape)
print(X_test4.shape)
print(y_test4.shape)
print(X_test5.shape)
print(y_test5.shape)


(2976, 8)
(2976, 2)
(2232, 8)
(2232, 2)
(744, 8)
(744, 2)
(2232, 8)
(2232, 2)
(2976, 8)
(2976, 2)
----
(2380, 8)
(2380, 2)
(1785, 8)
(1785, 2)
(595, 8)
(595, 2)
(1785, 8)
(1785, 2)
(2380, 8)
(2380, 2)
----
(596, 8)
(596, 2)
(447, 8)
(447, 2)
(149, 8)
(149, 2)
(447, 8)
(447, 2)
(596, 8)
(596, 2)


In [10]:
# Write data in CSV files
train1.to_csv("clean_data/train1_" + DATA + ".csv", index = False)
train2.to_csv("clean_data/train2_" + DATA + ".csv", index = False)
train3.to_csv("clean_data/train3_" + DATA + ".csv", index = False)
train4.to_csv("clean_data/train4_" + DATA + ".csv", index = False)
train5.to_csv("clean_data/train5_" + DATA + ".csv", index = False)

y_1.to_csv("clean_data/y_1.csv", index = False)
y_2.to_csv("clean_data/y_2.csv", index = False)
y_3.to_csv("clean_data/y_3.csv", index = False)
y_4.to_csv("clean_data/y_4.csv", index = False)
y_5.to_csv("clean_data/y_5.csv", index = False)

X_train1.to_csv("clean_data/X_train1_" + DATA + ".csv", index = False)
X_train2.to_csv("clean_data/X_train2_" + DATA + ".csv", index = False)
X_train3.to_csv("clean_data/X_train3_" + DATA + ".csv", index = False)
X_train4.to_csv("clean_data/X_train4_" + DATA + ".csv", index = False)
X_train5.to_csv("clean_data/X_train5_" + DATA + ".csv", index = False)

y_train1.to_csv("clean_data/y_train1.csv", index = False)
y_train2.to_csv("clean_data/y_train2.csv", index = False)
y_train3.to_csv("clean_data/y_train3.csv", index = False)
y_train4.to_csv("clean_data/y_train4.csv", index = False)
y_train5.to_csv("clean_data/y_train5.csv", index = False)

X_test1.to_csv("clean_data/X_test1_" + DATA + ".csv", index = False)
X_test2.to_csv("clean_data/X_test2_" + DATA + ".csv", index = False)
X_test3.to_csv("clean_data/X_test3_" + DATA + ".csv", index = False)
X_test4.to_csv("clean_data/X_test4_" + DATA + ".csv", index = False)
X_test5.to_csv("clean_data/X_test5_" + DATA + ".csv", index = False)

y_test1.to_csv("clean_data/y_test1.csv", index = False)
y_test2.to_csv("clean_data/y_test2.csv", index = False)
y_test3.to_csv("clean_data/y_test3.csv", index = False)
y_test4.to_csv("clean_data/y_test4.csv", index = False)
y_test5.to_csv("clean_data/y_test5.csv", index = False)

test.to_csv("clean_data/test_" + DATA + ".csv", index = False)